In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pymetis
import networkx as nx
import time
from networkx.algorithms import community
from random import shuffle
import math
import torch
import torch.nn as nn
import torch_geometric as tg
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree
from torch.nn import init
import pdb
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data
import torch.optim as optim
import pywt
from scipy.stats import norm
import scipy.interpolate as interp

In [2]:
def open_data(file_path):
    file = open(file_path,"rb")
    raw_data = pickle.load(file)  
    return raw_data

In [3]:
class NonLinearOperate(nn.Module):
    def __init__(self, input_dimen, hidden_dimen, output_dimen):
        super(NonLinearOperate, self).__init__()  #类NonLinearLayer继承父类nn.Module的初始化方法
        self.layer_1 = nn.Linear(input_dimen, hidden_dimen)
        self.layer_2 = nn.Linear(hidden_dimen, output_dimen)
        self.acti_func = nn.ReLU()
        for m in self.modules():#遍历所有子模块
#Check if each sub-module is an example of the class nn.Linear
            if isinstance(m, nn.Linear):
                m.weight.data = init.xavier_uniform_(m.weight.data, gain=nn.init.calculate_gain('relu'))#Use Xavier initialization
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0)
            
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.acti_func(x)
        x = self.layer_2(x)
        
        return x

In [4]:
class PGNN_Layer(nn.Module):
    def __init__(self, input_dimen, output_dimen, max_ach_num):
        super(PGNN_Layer, self).__init__()
        self.input_dimen = input_dimen
        self.output_dimen = output_dimen
        self.distance_calculate = NonLinearOperate(1, output_dimen, 1)
        self.acti_func = nn.ReLU()
        self.linear_hidden = nn.Linear(2*input_dimen, output_dimen)
        self.out_transition = nn.Linear(output_dimen,1)
        self.linear_out_position = nn.Linear(max_ach_num,input_dimen)
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data = init.xavier_uniform_(m.weight.data, gain=nn.init.calculate_gain('relu'))#Use Xavier initialization
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0)                
                    
    def forward(self, node_features, dists_max, dists_argmax):
        dists_max = self.distance_calculate(dists_max.unsqueeze(-1)).squeeze()
        subset_features = node_features[dists_argmax.flatten(), :]
        subset_features = subset_features.reshape(dists_argmax.shape[0], dists_argmax.shape[1], subset_features.shape[1])
        messages = subset_features * dists_max.unsqueeze(-1)
        feature_self = node_features.unsqueeze(1).repeat(1, dists_max.shape[1],1)
        messages = torch.concat((messages, feature_self), dim = -1) #N行M列D维
        messages = self.linear_hidden(messages).squeeze()#将输出维度改为Output Dimen,即n*m*output_dimen
        messages = self.acti_func(messages) 
        output_transition = self.out_transition(messages).squeeze(-1) #n * m * output_dimen to n * m
        output_position = self.linear_out_position(output_transition)
        output_structure = torch.mean(messages, dim=1)#n*output_dimen
        
        return output_position, output_structure

In [5]:
class PGNN(nn.Module):
    def __init__(self, input_dimen, hidden_dimen, output_dimen, max_ach_num, layer_num = 1, drop_out = True):
        super(PGNN, self).__init__()
        self.drop_out = drop_out
        self.layer_num = layer_num
        self.input_layer = nn.Linear(input_dimen, hidden_dimen)
        self.last_layer = nn.Linear(hidden_dimen, input_dimen)
        self.max_ach_num = max_ach_num
        if self.layer_num == 1:
            self.gnn_operate_1 = PGNN_Layer(hidden_dimen, output_dimen, max_ach_num)#输出维度是node_num * hidden_dimen
            
        if self.layer_num > 1:
            self.gnn_hidden = nn.ModuleList([PGNN_Layer(hidden_dimen, hidden_dimen, max_ach_num) for i in range(0, layer_num)])
            self.gnn_output_layer = PGNN_Layer(hidden_dimen, output_dimen)
                
        
    def forward(self, x, dist_max_sets, dist_argmax_sets):
        
        x = self.input_layer(x)
        if self.layer_num == 1:
            x_position, x = self.gnn_operate_1(x, dist_max_sets[0,:,:], dist_argmax_sets[0,:,:])
            if self.drop_out:
                x = F.dropout(x, training=self.training)
            x_position = self.last_layer(x_position)
            #print(f"pgnn output_layer size: {x_position.shape}")
            return x_position
    
        if self.layer_num > 1:
            for i in range(self.layer_num):
                _, x = self.gnn_hidden[i](x, dist_max_sets[i,:,:], dist_argmax_sets[i,:,:])
                if self.drop_out:
                    x = F.dropout(x, training=self.training)
                  
            _ = F.normalize(_, p=2, dim=-1)
            x_position = self.last_layer(_)    
            
            
            return x_position

In [6]:
class P_GCN(nn.Module):
    def __init__(self, input_dimen, hidden_dimen, output_dimen, max_ach_num, layer_num = 1, drop_out = True):
        super(P_GCN, self).__init__()
        self.max_ach_num = max_ach_num
        self.layer_num = layer_num
        self.drop_out = drop_out
        self.input_layer = nn.Linear(input_dimen, hidden_dimen)
        self.p_gcn_block = nn.Sequential(PGNN_Layer(hidden_dimen, hidden_dimen, max_ach_num), 
                                         GCNConv(hidden_dimen, hidden_dimen, add_self_loops=True))
        self.acti_func = nn.ReLU()   
        
        if layer_num == 1:
            self.gcn_p_layers = self.p_gcn_block
        if layer_num > 1:
            self.gcn_p_layers = nn.ModuleList([self.p_gcn_block for i in range(num_layer)])
        self.output_layer = nn.Linear(hidden_dimen, output_dimen)
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data = init.xavier_uniform_(m.weight.data, gain=nn.init.calculate_gain('relu'))
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0)   
        
        
    def forward(self, x, edge_index, dist_max, dist_argmax):#GCN_P_input shape: node_num, 4
        
        x_ = self.input_layer(x)
        
        if self.layer_num == 1:
            x_position, _ = self.gcn_p_layers[0](x_, dist_max[0,:,:], dist_argmax[0,:,:])
            if self.drop_out:
                x_position = F.dropout(x_position, training=self.training)
            x = self.gcn_p_layers[1](x_position, edge_index)
            if self.drop_out:
                x = F.dropout(x, training=self.training)
            x = self.acti_func(x + x_)
            
        else:
            for i in range(self.layer_num):
                x_position, _ = self.gcn_p_layers[i][0](x_, dist_max[i,:,:], dist_argmax[i,:,:])
                if self.drop_out:
                    x_position = F.dropout(x_position, training=self.training)
                x = self.gcn_p_layers[i][1](x_position, edge_index)
                if self.drop_out:
                    x = F.dropout(x, training=self.training)
                x = self.acti_func(x + x_)
                x_ = x
                    
        x = self.acti_func(self.output_layer(x))
    
        return x

In [7]:
#输入数据shape: node_num, 4, time_step(20)
class CNN_1D(nn.Module):
    def __init__(self, input_channels, hidden_channels_1, hidden_channels_2, out_channels):
        super(CNN_1D, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels = input_channels, out_channels = hidden_channels_1, kernel_size = 3, padding=1),
            nn.BatchNorm1d(hidden_channels_1),
            nn.ReLU(),
            nn.Conv1d(in_channels = hidden_channels_1, out_channels = hidden_channels_2, kernel_size = 3, padding=0),
            nn.BatchNorm1d(hidden_channels_2),
            nn.ReLU(), #len: 18
            nn.MaxPool1d(kernel_size=2, stride=2),#(18-2)/2 +1 =9
            nn.ConvTranspose1d(in_channels= hidden_channels_2,
                               out_channels=out_channels,
                               kernel_size=4,
                               stride= 2, 
                               padding=0))

        self.fc1 = nn.Linear(out_channels, 4)
    
    def forward(self, x):
        out = self.layer1(x)
        #output shape: change from batch_num, out_channels, t-step to batch_num, t-step, out_channels
        out = out.permute(0, 2, 1)
        out = self.fc1(out)
        #output shape: batch_num, t-step, 4
        return out

In [8]:
class GRU(nn.Module):
    def __init__(self, input_dimen, hidden_dimen, pred_len, output_dimen = 4, num_layers = 2):
        super(GRU, self).__init__()
        self.hidden_dimen = hidden_dimen
        self.output_dimen = output_dimen
        self.num_layers = num_layers
        self.linear_layer_1 = nn.Linear(input_dimen, hidden_dimen)
        self.gru_layers = nn.GRU(hidden_dimen, hidden_dimen, num_layers, batch_first = True)
        self.linear_layer_2 = nn.Linear(hidden_dimen, output_dimen)
        self.k = pred_len #Futre k-step to predict 
        
         
    #gru输入格式：node_num, t-steps, hidden_dimen
    def forward(self, pgnn_t_step_outs, extractor_outputs):  
        x = torch.cat((pgnn_t_step_outs, extractor_outputs), dim = -1)
        batch_size, seq_len, feat_dim = x.size()
        h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_dimen)
        x = self.linear_layer_1(x)
        outputs, _ = self.gru_layers(x, h_0)#outputs shape: batch_size, seq_len, hidden dim. 
        layer_2_input = outputs[:, -self.k:, :]
        x = self.linear_layer_2(layer_2_input) #x shape: batch_size, k steps, 4.
        #linear_layer_2 trasnfer last dimension from hidden dim to 4
        
        return x

In [9]:
class T_Step_PGNN(nn.Module):
    def __init__(self, input_dimen, hidden_dimen, output_dimen, max_ach_num, layer_num = 1):
        super(T_Step_PGNN, self).__init__()
        self.pgnn_model = PGNN(input_dimen, hidden_dimen, output_dimen, max_ach_num)
        
    def forward(self, p_gnn_input_sets, subgraph_nodes, dist_max, dist_argmax): 
        p_gnn_template = torch.zeros((p_gnn_input_sets.shape[0], p_gnn_input_sets.shape[1], 4))
        indices = [torch.isin(p_gnn_input_sets[:, :, 0], graph_node).unsqueeze(-1).expand_as(p_gnn_template) for graph_node in subgraph_nodes]
        for i in range(len(indices)):
            subgraph = p_gnn_input_sets[:,:,1:5][indices[i]].view(p_gnn_input_sets.shape[0], -1, 4)
            p_gnn_outputs = torch.empty((0,subgraph.shape[1],subgraph.shape[2]))
            subgraph_node_num = subgraph_nodes[i].shape[0]
            
            for t in range(subgraph.shape[0]):
                p_gnn_t_step = self.pgnn_model(subgraph[t,:,:], dist_max[i,:,:subgraph_node_num,:], dist_argmax[i,:,:subgraph_node_num,:])
                p_gnn_outputs = torch.cat((p_gnn_outputs, p_gnn_t_step.unsqueeze(0)), dim=0)
            
            p_gnn_template[indices[i]] = p_gnn_outputs.view(-1)
        
        return pgcn_template

In [12]:
class Feature_Extractor(nn.Module):
    def __init__(self, input_dimen, hidden_dimen, output_dimen, max_ach_num,
                input_channels, hidden_channels_1, hidden_channels_2, out_channels, layer_num = 1):
        super(Feature_Extractor, self).__init__()
        self.pgcn_model = P_GCN(input_dimen, hidden_dimen, output_dimen, max_ach_num)
        self.cnn_1D = CNN_1D(input_channels, hidden_channels_1, hidden_channels_2, out_channels)
        
        
    def forward(self, p_gcn_input_sets, edge_index, subgraph_nodes, dist_max, dist_argmax): 
        pgcn_template = torch.zeros((p_gcn_input_sets.shape[0], p_gcn_input_sets.shape[1], 4))
        indices = [torch.isin(p_gcn_input_sets[:, :, 0], graph_node).unsqueeze(-1).expand_as(pgcn_template) for graph_node in subgraph_nodes]
       
            
        for i in range(len(indices)):
            subgraph = p_gcn_input_sets[:,:,1:5][indices[i]].view(p_gcn_input_sets.shape[0], -1, 4)
            print(subgraph.shape[1])
            p_gcn_outputs = torch.empty((0,subgraph.shape[1],subgraph.shape[2]))
            subgraph_node_num = subgraph_nodes[i].shape[0]
            
            
            for t in range(subgraph.shape[0]):
                p_gcn_t_step = self.pgcn_model(subgraph[t,:,:], edge_index[i], dist_max[i,:,:subgraph_node_num,:], dist_argmax[i,:,:subgraph_node_num,:])
                p_gcn_outputs = torch.cat((p_gcn_outputs, p_gcn_t_step.unsqueeze(0)), dim=0)
            
            pgcn_template[indices[i]] = p_gcn_outputs.view(-1)
            del p_gcn_outputs
            
        #shape changed as: node_num, 4, time_step
        pgcn_template =pgcn_template.permute(1,2,0)
        extractor_outputs = self.cnn_1D(pgcn_template).permute(1, 0, 2)
        #cnn_output shape changed from node_num, t-step, 4 to t-step, node_num, 4 
        
        return extractor_outputs

In [13]:
def sort_pooling(extractor_outputs, k):
    norms = torch.norm(extractor_outputs, p=2, dim= -1)
    _, sorted_indices = torch.sort(norms, dim= -1, descending=True)
    sorted_outputs = torch.gather(extractor_outputs, dim=1, index=sorted_indices.unsqueeze(-1).expand(-1, -1, extractor_outputs.size(-1)))
    k_nodes_outputs = sorted_outputs[:, : k , :]
    
    return k_nodes_outputs

In [14]:
class Discriminator(nn.Module):
    def __init__(self, input_dimen, hidden_dimen, k, layer_num = 1):
        super(Discriminator, self).__init__()
        self.k = k
        self.layer_num = layer_num
        self.batch_norm = nn.BatchNorm1d(num_features = input_dimen)
        self.input_layer = nn.Linear(input_dimen, hidden_dimen)
    
        if layer_num == 1:
            self.linear_layers = nn.Linear(hidden_dimen, 1)
        else:
            self.linear_layers = nn.ModuleList([nn.Linear(hidden_dimen, hidden_dimen), nn.ReLU()] * (layer_num - 1))
            self.linear_layers.append(nn.Linear(hidden_dimen, 1))
        
        self.acti_func = nn.ReLU()
        self.classifier = nn.Sigmoid() 
        
    def forward(self, extractor_outputs):
        k_nodes_outputs = sort_pooling(extractor_outputs, self.k)
        norm_x = self.batch_norm(k_nodes_outputs)
        x = self.input_layer(norm_x)
        
        if self.layer_num == 1:
            x = self.linear_layers(self.acti_func(x))
        else:
            x = self.linear_layers(x)
        
        x = self.classifier(x)
        
        return x

In [15]:
def renumber_subgraph(nodes, edge_index):
    unique_nodes = torch.unique(nodes, sorted = True)
    new_node_ids = torch.arange(len(unique_nodes))
    node_mapping = {old_id.item(): new_id.item() for old_id, new_id in zip(unique_nodes, new_node_ids)}
    
    new_edge_index = torch.tensor([
        [node_mapping[edge_index[0, i].item()], node_mapping[edge_index[1, i].item()]]
        for i in range(edge_index.size(1))
    ]).t()
    
    return new_edge_index

In [16]:
class CustomData(Data):
    def __init__(self, trend, period, target_volume, target_label, edge_pairs, subgraph_node_num, subgraph_nodes,city_node_num, dist_max, dist_argmax):
        super(CustomData, self).__init__()
        self.trend = trend
        self.period = period
        self.target_volume = target_volume
        self.target_label = target_label
        self.edge_pairs = edge_pairs
        self.subgraph_node_num = subgraph_node_num
        self.subgraph_nodes = subgraph_nodes
        self.city_node_num = city_node_num
        self.dist_max = dist_max
        self.dist_argmax = dist_argmax

In [17]:
input_dimen, hidden_dimen, output_dimen = 4, 8 ,4
max_ach_num = 72
input_channels, hidden_channels_1, hidden_channels_2, out_channels = 4, 8, 16, 8
k = 60
pred_len = 10

In [18]:
#-----------------------PGNN----------------------------
pgnn_model = T_Step_PGNN(input_dimen, hidden_dimen, output_dimen, max_ach_num)
    
#-------------------Feature Extractor----------------------   
feature_extractor = Feature_Extractor(input_dimen, hidden_dimen, output_dimen, max_ach_num,
                                      input_channels, hidden_channels_1, hidden_channels_2, out_channels)

#-------------------Discriminator-----------------------------
discriminator = Discriminator(input_dimen, hidden_dimen, k, layer_num = 2)

#------------------Predictor-----------------------------------
predictor = GRU(input_dimen, hidden_dimen, pred_len)



In [19]:
criterion_discriminator = nn.BCELoss()
criterion_regression = nn.MSELoss(reduction='sum')


optimizer_extractor = optim.Adam(feature_extractor.parameters(), lr=0.0008)
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=0.0006)
optimizer_pgnn = optim.Adam(pgnn_model.parameters(), lr=0.0008)
optimizer_predictor = optim.Adam(predictor.parameters(), lr=0.0008)

In [20]:
purpose = "test"
batch_size = 4
root_path = "D:/ThesisData/processed data/"
file_path = root_path + f'/SourceDomain/loaders/{purpose}_list.pt'
data = torch.load(file_path)
batch_num = len(data)//batch_size

In [21]:
batch_num

7

In [ ]:
epoch_num = 1

for h in range(epoch_num):
    for b in range(batch_num):
        batch = data[b: b + batch_size]
        city_nodes = sum([batch[i].city_node_num for i in range(len(batch))])
        
        one_batch_discriminator = []
        one_batch_predictor = []
        target_volumes = []
        target_labels = []
        
        #Each Batch Covers batch_size data files
        for i in range(len(batch)):
    
            #extractor outputs Shape: T-step, Node_num, 4

            batch_id = batch[i].trend[0,:,0]
            sorted_batch_id, _ = torch.sort(batch_id)
            
            batch_nodes = torch.cat(batch[i].subgraph_nodes, dim = 0)
            sorted_batch_nodes, _ = torch.sort(batch_nodes)
            
            batch_id_ = batch[i].period[0,:,0]
            sorted_batch_id_, _ = torch.sort(batch_id_)
           
            
            print("sorted trend batch ID:")
            print(sorted_batch_id.shape[0])
            print(torch.min(sorted_batch_id))
            print("sorted period batch ID:")
            print(sorted_batch_id_.shape[0])
            print(torch.min(sorted_batch_id_))
            
            print("-----------------------")
            print("sorted batch nodes")
            print(sorted_batch_nodes.shape[0])
            print(torch.min(sorted_batch_nodes))
            
            
            extractor_outputs = feature_extractor(batch[i].trend, batch[i].edge_pairs, batch[i].subgraph_nodes, batch[i].dist_max, batch[i].dist_argmax)    
            
            
            pgnn_outputs = pgnn_model(batch[i].period, batch[i].subgraph_nodes, batch[i].dist_max, batch[i].dist_argmax)
                   
            discriminator_out = discriminator(extractor_outputs.detach())
            
                                      
            predictor_output = predictor(pgnn_outputs.permute(1, 0, 2), extractor_template.permute(1, 0, 2))
            predictor_output = predictor_output.permute(1, 0, 2)
                                      
            one_batch_predictor.append(predictor_output)
            one_batch_discriminator.append(discriminator_out)
            
            target_volumes.append(batch[i].target_volume)            
            target_labels.append(batch[i].target_label[:,:k])
                                      
        one_batch_predictor = torch.stack(one_batch_predictor)
        one_batch_discriminator = torch.stack(one_batch_discriminator)
        target_volumes = torch.stack(target_volumes)                  
        target_labels = torch.stack(target_labels)
                                      
                                      
        #--------Update PGNN Para-------------------
        optimizer_pgnn.zero_grad()
        loss_pgnn = criterion_regression(one_batch_predictor, target_volume) / city_nodes
        loss_pgnn.backward()
        optimizer_pgnn.step()

        
        #---------Update the Para of Discriminator-----------
        optimizer_discriminator.zero_grad()
        loss_discriminator = criterion_discriminator(one_batch_discriminator, target_labels)
        loss_discriminator.backward()
        optimizer_discriminator.step()


        #--------Update Feature Extractor Para-------------
        #--------Firstly, Use UPDATED DISCRIMINATOR TO CLASSIFIED DOMAIN DATA---------
        discriminator_out = discriminator(extractor_outputs)
        loss_discriminator = criterion_discriminator(discriminator_out, target_labels)
        #--------Secondly, update the PARAs of FEATURE EXTRACTOR------------
        optimizer_extractor.zero_grad()
        loss_feat_ext = alpha * loss_pgnn + (1-alpha) * (-loss_discriminator) 
        loss_feat_ext.backward()
        optimizer_extractor.step()


        #---------Update GRU Paras AND NOT INFLUENCE PGNN--------
        optimizer_predictor.zero_grad()
        gru_k_step_outs = predictor(pgnn_outputs.permute(1, 0, 2).detach(), extractor_template.permute(1, 0, 2).detach()).permute(1, 0, 2)
        loss_gru = criterion_regression(gru_k_step_outs, target_volumes)/ city_nodes
        loss_gru.backward()
        optimizer_predictor.step()

        print(f"After batch {b}, regression loss: {loss_gru}, disriminator loss: {loss_discriminator}")
        


sorted trend batch ID:
111260
tensor(0.)
sorted period batch ID:
111260
tensor(0.)
-----------------------
sorted batch nodes
91160
tensor(13)
3617
4693
3873
3736
3992
4011
4565
4430
4165
4704
4795
4099
4826
4148
3755
3858
3240
4129
3587
4131
4257
4549
